In [3]:
import os
import numpy as np
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.ml.linalg import SparseVector
from scipy.sparse import load_npz


os.environ['PYSPARK_PYTHON'] = 'C:\ProgramData\mambaforge\envs\ML-base\python.exe'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'C:\ProgramData\mambaforge\envs\ML-base\Scripts\ipython.exe'

spark = SparkSession.builder.appName("MyApp").getOrCreate()
sc=spark.sparkContext
spark

c:\ProgramData\mambaforge\envs\ML-base\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
df_id=pd.read_parquet("ids_nfcorpus.parquet")
#vocab=pd.read_parquet("terms_nfcorpus.parquet")
threshold=0.8

In [4]:
sparse_repr=load_npz("sparse_repr_nfcorpus.npz")

In [53]:
def csr_to_sparse_vector(row):
    return SparseVector(row.shape[1], list(zip(row.indices, row.data)))

docs = [csr_to_sparse_vector(sparse_repr.getrow(i)) for i in range(sparse_repr.shape[0])]
doc_ids = df_id["_id"]

In [149]:
def sparse_argsort(matrix, idx):
    row=matrix.getrow(idx)
    sorted_indices = np.argsort(row.data)[::-1]
    return row.indices[sorted_indices]
sorted_index_term_doc=[sparse_argsort(sparse_repr, idx) for idx in range(sparse_repr.shape[0])]

In [152]:
#For debugging
#np.max(sparse_repr.getcol(0))
#np.max(sparse_repr, axis=0).toarray()

In [153]:
d_star=np.max(sparse_repr, axis=0).toarray().reshape(-1)
b_d=[]
for row in sparse_repr:
    mult = np.multiply(row.toarray(), d_star)
    cum_sum=np.cumsum(mult)
    #print(cum_sum.asarray())
    index = np.argmax(mult < threshold)
    b_d.append(cum_sum[index]) #b(d) value """
b_d=np.array(b_d)

In [154]:
sparse_repr.getrow(0).toarray().flatten()[0]

0.041906272848198825

In [156]:
rdd_forMap=sc.parallelize([(doc_ids[i], (docs[i], sorted_index_term_doc[i], b_d[i])) for i in range(sparse_repr.shape[0])])

In [157]:
rdd_forMap.first()

('MED-10',
 (SparseVector(18867, {0: 0.0419, 56: 0.1441, 153: 0.1034, 389: 0.1581, 601: 0.1235, 629: 0.0422, 638: 0.1093, 741: 0.1735, 768: 0.0741, 909: 0.046, 940: 0.0898, 1062: 0.093, 1119: 0.0518, 1183: 0.0965, 1218: 0.0997, 1346: 0.0995, 1358: 0.0938, 1437: 0.0535, 1438: 0.082, 1463: 0.1645, 1525: 0.1011, 1825: 0.0642, 1886: 0.0599, 1947: 0.024, 2173: 0.1064, 2186: 0.0686, 2247: 0.0734, 2264: 0.0519, 2281: 0.0516, 2408: 0.0475, 2446: 0.1229, 2523: 0.0598, 2555: 0.0396, 2592: 0.0223, 2982: 0.0353, 3033: 0.0428, 3060: 0.0436, 3334: 0.0612, 3399: 0.0374, 3429: 0.0767, 3462: 0.0773, 3532: 0.1208, 3791: 0.0559, 3858: 0.0792, 3974: 0.0498, 4000: 0.0333, 4117: 0.0564, 4203: 0.0707, 4243: 0.0634, 4288: 0.1153, 4497: 0.0944, 4667: 0.0653, 4782: 0.1145, 4792: 0.0611, 4896: 0.0753, 5269: 0.0717, 5302: 0.0945, 5609: 0.0941, 5648: 0.0794, 5692: 0.0601, 5750: 0.1075, 5819: 0.0769, 5937: 0.0918, 5938: 0.0913, 5939: 0.1137, 5997: 0.1096, 6138: 0.1295, 6151: 0.0481, 6322: 0.0744, 6390: 0.0819, 6414

Map( (doc_id, (doc, sorted_index_term_doc, b_d)) )
    return (index_term, (doc_id, doc) )

GroupBy: done by Spark

Reduce( (index_term,  list( (doc_id, doc) )) )



In [166]:
docs[0][741]

0.17349529008931233

In [169]:
sorted_index_term_doc[0][0]

741

In [167]:
b_d[0]

0.003145170143285335

In [163]:
def my_map(elem):
    for t_idx in elem[1][1]:
        if elem[1][0][t_idx]>elem[1][2]:
            return (t_idx, (elem[0], elem[1][0]))
tmp=rdd_forMap.map(my_map)

In [164]:
tmp.first()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 27.0 failed 1 times, most recent failure: Lost task 0.0 in stage 27.0 (TID 94) (192.168.1.56 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\ProgramData\mambaforge\envs\ML-base\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 830, in main
  File "C:\ProgramData\mambaforge\envs\ML-base\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 822, in process
  File "C:\ProgramData\mambaforge\envs\ML-base\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "c:\ProgramData\mambaforge\envs\ML-base\lib\site-packages\pyspark\rdd.py", line 2830, in takeUpToNumLeft
    yield next(iterator)
  File "C:\ProgramData\mam

In [ ]:
spark.stop()